In [ ]:
import os
if any("COLAB" in e for e in os.environ):
    !pip install pykrx

# INITIALIZE

In [1]:
from datetime import datetime, timedelta
from pytz import timezone
from pykrx import stock
import pandas as pd


get_td = stock.get_nearest_business_day_in_a_week
get_mk = stock.get_market_cap_by_ticker
get_fr = stock.get_exhaustion_rates_of_foreign_investment_by_ticker


TD = get_td()
if 900 <= int(datetime.now(timezone('Asia/Seoul')).strftime("%H%M")) <= 1530:
    TD = get_td((datetime.today() - timedelta(1)).strftime("%Y%m%d"))

TD_SET = {
    f'D-{n}': get_td((datetime.strptime(TD, "%Y%m%d") - timedelta(n)).strftime("%Y%m%d"))
    for n in [1, 7, 14, 30, 61, 92, 183, 365]
}

In [ ]:
print(f"TRADING DATE: {TD}")
for k, v in TD_SET.items():
    print(f"D-{k.split('-')[-1].zfill(3)}: {v}")

# GENERALS

In [2]:
mk = pd.concat([get_mk(TD, market="KOSPI"), get_mk(TD, market="KOSDAQ")])
fr = pd.concat([get_fr(TD, market="KOSPI"), get_fr(TD, market="KOSDAQ")])
fr = fr.drop(columns=[c for c in fr if c in mk])
GENERALS = pd.concat([mk, fr], axis=1).astype('float64')

In [ ]:
# GENERALS
# GENERALS.columns

# FOREIGN RATE

In [19]:
objs = {key:get_fr(date, market="ALL")['지분율'] for key, date in TD_SET.items()}
frs = pd.concat(objs, axis=1)
frs = pd.concat([GENERALS['지분율'], frs], axis=1).astype('float64')
frs = frs[(~frs['지분율'].isna()) & (frs['지분율'] != 0)]
frs = frs[frs.columns.tolist()[::-1]]
frs = frs.rename(columns={'지분율': 'D-0'})
# frs

## Delta

In [22]:
_delta = {}
for col in frs:
    if col == 'D-0':
        break
    _delta[f'{col}[delta]'] = frs['D-0'] - frs[col]
delta = pd.concat(_delta, axis=1)
# delta

## Regression

In [23]:
from scipy.stats import linregress

def calculate_trend_strength(series):
    if series.isnull().all() or (series == 0).all():
        return 0.0

    x = range(len(series))
    y = series.values

    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    return slope

columns = frs.columns.tolist()
_slope = {}
for n, c in enumerate(columns):
    if c == "D-0":
        break
    _slope[f'{c}[slope]'] = frs[columns[n:]].apply(calculate_trend_strength, axis=1)
slope = pd.concat(_slope, axis=1)

In [24]:
df = frs.join(delta, how='left').join(slope, how='left')
df.to_csv('foreign_rate.csv')

In [ ]:
from pykrx import stock

# Get ticker list for KOSPI and KOSDAQ
kospi_tickers = stock.get_market_ticker_list(TD, market="KOSPI")
kosdaq_tickers = stock.get_market_ticker_list(TD, market="KOSDAQ")

# Combine ticker lists
all_tickers = kospi_tickers + kosdaq_tickers

# Get stock names for all tickers
stock_names = [stock.get_market_ticker_name(ticker) for ticker in all_tickers]

# Display the list of stock names
print("전체 종목명 리스트:")
for name in stock_names:
    print(name)